<a href="https://colab.research.google.com/github/Tahahaha7/Quantitative_Trading/blob/main/Securities_and_Exchange_Commission_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%%capture
!pip -q install wget yfinance
!pip install ratelimit==2.2.0

In [ ]:
import wget
wget.download('https://gist.githubusercontent.com/neila/0184545e7a643ff7c506673b15ac3c2b/raw/3efd37ccb3b04513587b1de06494ea6f17aecaad/project_helper.py', 
              out="project_helper.py")

In [22]:
import urllib
import nltk
import numpy as np
import pandas as pd
import re, requests
import pickle
import json
import pprint
import project_helper
from tqdm import tqdm

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [37]:
def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}

    for ticker in TICKERS:
        get_request = requests.get(URL.format(ticker), stream=True);
        results = CIK_RE.findall(get_request.text)
        try: cik_dict[ticker] = results[0]
        except: pass
    print('Number of stocks retrieved {}'.format(len(results)))
    return cik_dict

assert getCIKs(['AMZN','BMY','CNP','CVX','FL','FRT','HON']) == {'AMZN': '0001018724','BMY': '0000014272',
                                                                'CNP': '0001130310','CVX': '0000093410',
                                                                'FL': '0000850209','FRT': '0000034903', 
                                                                'HON': '0000773840'}

Number of stocks retrieved 7


In [10]:
# A set of random stock tickers
link = 'https://raw.githubusercontent.com/RayBB/random-stock-picker/master/stocks.json'
response = urllib.request.urlopen(link)
stocks = json.loads(response.read())

In [38]:
# Some tickers are not returning any results
stockchoices = np.random.choice(stocks, 20, replace=False)
cik_lookup = getCIKs(stockchoices)

Number of stocks retrieved 6


In [39]:
sec_api = project_helper.SecAPI()

In [40]:
from bs4 import BeautifulSoup

def get_sec_data(cik, doc_type, start=0, count=60):

    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom'\
    .format(cik, doc_type, start, count)

    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'xml').feed
    entries = [(entry.content.find('filing-href').getText(),
                entry.content.find('filing-type').getText(),
                entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)]
    return entries

In [41]:
sec_data = {}

for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')

example_ticker = stockchoices[0]
print("example ticker:", example_ticker)
pprint.pprint(sec_data[example_ticker][:5])

example ticker: FONR
[('https://www.sec.gov/Archives/edgar/data/355019/000035501920000051/0000355019-20-000051-index.htm',
  '10-K',
  '2020-10-01'),
 ('https://www.sec.gov/Archives/edgar/data/355019/000035501919000036/0000355019-19-000036-index.htm',
  '10-K',
  '2019-09-30'),
 ('https://www.sec.gov/Archives/edgar/data/355019/000035501918000039/0000355019-18-000039-index.htm',
  '10-K',
  '2018-09-21'),
 ('https://www.sec.gov/Archives/edgar/data/355019/000035501917000029/0000355019-17-000029-index.htm',
  '10-K',
  '2017-09-27'),
 ('https://www.sec.gov/Archives/edgar/data/355019/000035501916000059/0000355019-16-000059-index.htm',
  '10-K',
  '2016-09-28')]
